In [39]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [40]:
path = '../data/'
savepath = '../data/'
images = glob.glob(path+'new_data/*.npy')

In [41]:
tile_sizes = np.loadtxt(path+'tile_sizes.txt', dtype='int')
tile_sizes = tile_sizes[:150]
images_sampled = {}
for tile in tile_sizes:
    for i in range(1, 31):
        images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])

In [42]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/nparrs_384/frame'+str(i)+'.npy'
            frame = np.load(frame)
            tile_index = random.choice(images_sampled[i])
            
            temp  = imresize(frame[tile_index], (48, 48))
            temp  = imresize(temp, (384, 384))
            
            alldata_x.append(temp)
            alldata_y.append(frame[tile_index])
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        alldata_x = alldata_x.astype(np.float32)/255.0
        
        yield alldata_x, alldata_y

#x = myGenerator(10)
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [43]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        input_size = (384,384,3)
        inputs = Input(input_size)


        conv1 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = LeakyReLU()(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)



        conv2 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv2 = LeakyReLU()(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv3 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv3 = LeakyReLU()(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        concat1 = add([conv1, conv3])



        conv4 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat1)
        conv4 = LeakyReLU()(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        conv5 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv5 = LeakyReLU()(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        concat2 = add([conv5, concat1])



        conv6 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat2)
        conv6 = LeakyReLU()(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        conv7 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
        conv7 = LeakyReLU()(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        concat3 = add([conv7, concat2])



        conv8 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat3)
        conv8 = LeakyReLU()(conv8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)

        conv9 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv9 = LeakyReLU()(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        

        conv10 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = LeakyReLU()(conv10)
        conv10 = BatchNormalization(momentum=0.8)(conv10)

        concat4 = add([conv10, conv1])
        
        conv11 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat4)
        conv11 = LeakyReLU()(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)
        
        conv12 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv11)
        conv12 = LeakyReLU()(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)


        out = Conv2D(3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv12)
        out = LeakyReLU()(out)
        out = BatchNormalization(momentum=0.8)(out)


        model = Model(input = inputs, output = out)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, y_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        y_train = (0.5 * y_train + 0.5)*255
        y_train = y_train.astype(int)
        X_train = X_train*255
        X_train = X_train.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5], y_train[6], y_train[7], y_train[8], y_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_lowres.png", combined)

In [ ]:
cgan = CGAN()
cgan.build_autoencoder()
cgan.train_generator_autoencoder(200000, 10, 100)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:116: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
conv2d_86 (Conv2D)              (None, 384, 384, 8)  224         input_15[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_72 (LeakyReLU)      (None, 384, 384, 8)  0           conv2d_86[0][0]                  
__________________________________________________________________________________________________
batch_normalization_86 (BatchNo (None, 384, 384, 8)  32          leaky_re_lu_72[0][0]             
__________________________________________________________________________________________________
conv2d_87 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


Epoch  0  G loss  1.0184677


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:146: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:150: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:154: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Epoch  1  G loss  0.92984235
Epoch  2  G loss  0.956529
Epoch  3  G loss  0.87251264
Epoch  4  G loss  0.830764
Epoch  5  G loss  0.8966212
Epoch  6  G loss  0.7456883
Epoch  7  G loss  0.9110287
Epoch  8  G loss  0.832887
Epoch  9  G loss  0.76020443
Epoch  10  G loss  0.76627403
Epoch  11  G loss  0.6345604
Epoch  12  G loss  0.62804264
Epoch  13  G loss  0.6513373
Epoch  14  G loss  0.58473486
Epoch  15  G loss  0.5457722
Epoch  16  G loss  0.68295324
Epoch  17  G loss  0.49628335
Epoch  18  G loss  0.51771486
Epoch  19  G loss  0.6854014
Epoch  20  G loss  0.48135194
Epoch  21  G loss  0.48095575
Epoch  22  G loss  0.63420326
Epoch  23  G loss  1.1755085
Epoch  24  G loss  0.5220529
Epoch  25  G loss  0.62801194
Epoch  26  G loss  0.56874245
Epoch  27  G loss  0.44128904
Epoch  28  G loss  0.42979375
Epoch  29  G loss  0.35457775
Epoch  30  G loss  0.68591493
Epoch  31  G loss  0.4717962
Epoch  32  G loss  0.34736258
Epoch  33  G loss  0.42909685
Epoch  34  G loss  0.4223103
Epoch 

Epoch  272  G loss  0.23033924
Epoch  273  G loss  0.18398845
Epoch  274  G loss  0.20947488
Epoch  275  G loss  0.21128294
Epoch  276  G loss  0.3686301
Epoch  277  G loss  0.26126575
Epoch  278  G loss  0.14719217
Epoch  279  G loss  0.16539517
Epoch  280  G loss  0.22623697
Epoch  281  G loss  0.32053897
Epoch  282  G loss  0.39238504
Epoch  283  G loss  0.19520235
Epoch  284  G loss  0.2595955
Epoch  285  G loss  0.2314289
Epoch  286  G loss  0.1714792
Epoch  287  G loss  0.17484184
Epoch  288  G loss  0.30884057
Epoch  289  G loss  0.20676902
Epoch  290  G loss  0.15193188
Epoch  291  G loss  0.18033013
Epoch  292  G loss  0.18268092
Epoch  293  G loss  0.12966941
Epoch  294  G loss  0.18959984
Epoch  295  G loss  0.1831527
Epoch  296  G loss  0.203157
Epoch  297  G loss  0.35752526
Epoch  298  G loss  0.26065567
Epoch  299  G loss  0.31120303
Epoch  300  G loss  0.19355458
Epoch  301  G loss  0.2070487
Epoch  302  G loss  0.27819154
Epoch  303  G loss  0.17025657
Epoch  304  G lo

In [31]:
for i in range(1000):
    x = myGenerator(1)
    xtest, ytest = next(x)
    pred = cgan.generator.predict(xtest)
    pred = pred*127.5 + 127.5
    pred = pred.astype(int)
    #plt.imshow(pred[0])
    #plt.show()
    ytest = ytest*127.5+127.5
    ytest = ytest.astype(int)
    #plt.imshow(ytest[0])
    #plt.show()
    imsave(path+'test/frame_pred'+str(i)+'.png', pred[0])
    imsave(path+'test/frame_real'+str(i)+'.png', ytest[0])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [32]:
import numpy 
import math
import cv2

def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [36]:
data = []
for i in range(1000):
    img1 = cv2.imread(path+'test/frame_real'+str(i)+'.png')
    img2 = cv2.imread(path+'test/frame_pred'+str(i)+'.png')
    d = psnr(img1, img2)
    data.append(d)
print('PSNR:', np.mean(data))

PSNR: 31.612699630864867
